In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [61]:
from glob import glob
import random
import json
import pydicom
import pickle
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

DICOM_TAGS_DF_PATH = '/kolos/m2/ct/data/rsna/df.pkl'
LABELS_PATH = '/kolos/storage/ct/data/rsna/stage_1_train.csv'

DISEASES = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']

NUM_FOLDS = 5

In [62]:
csv_root_dir = os.path.normpath(os.getcwd() + '/../csv')

In [63]:
folds_orig = pd.read_csv(os.path.join(csv_root_dir, '5fold.csv'))
folds_corrected = pd.read_csv(os.path.join(csv_root_dir, '5fold-rev3.csv'))
id_map = pd.read_csv(os.path.join(csv_root_dir, 'id_map.csv'))
id_map = id_map.set_index('SOPInstanceUID')

In [64]:
folds_patients_test = pd.read_csv(os.path.join(csv_root_dir, 'fold_patients_test.csv'))

In [65]:
def load_labels(PATH):
    labels = pd.read_csv(PATH)
    labels[['SOPInstanceUID', 'Disease']] = labels.ID.str.rsplit("_", 1, expand=True)
    labels = labels[['SOPInstanceUID', 'Disease', 'Label']]
    labels = pd.pivot_table(labels, index="SOPInstanceUID", columns="Disease", values="Label")

    return labels

# TODO read given csv with labels for test set
test_csv_path = '/home/sp/repos/kaggle-rsna-2019/rsna19/data/csv/tt.csv'
test_labels_df = load_labels(test_csv_path)

In [66]:
folds_patients_test = folds_patients_test[['SOPInstanceUID', 'fold']]
folds_patients_test = folds_patients_test.set_index('SOPInstanceUID')
folds_patients_test

,fold
SOPInstanceUID,
ID_53e0c2d04,4
ID_ea32d6d6d,1
ID_434cabb4c,4
ID_22b3b726f,2
ID_cc10b6f55,1
...,...
ID_3579e1d1f,0
ID_bd7aaac7e,0
ID_8b23db3f4,2


In [74]:
folds_test = pd.merge(test_labels_df, folds_patients_test, on=['SOPInstanceUID'])
folds_test = pd.merge(folds_test, id_map, on=['SOPInstanceUID'])
folds_test

,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,fold,study_id,slice_num
SOPInstanceUID,,,,,,,,,
ID_53e0c2d04,0,0,0,0,0,0,4,ID_a7f5df3dae,5


In [75]:
folds_test['path'] = folds_test.apply(lambda x : f'rsna/test/{x.study_id}/npy/{x.slice_num:03d}.npy', axis=1)
folds_test = folds_test[['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'path', 'fold']]
folds_test = folds_test.reset_index(drop=True)
folds_test

,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,path,fold
0,0,0,0,0,0,0,rsna/test/ID_a7f5df3dae/npy/005.npy,4


In [80]:
folds_orig_test_appended = pd.concat([folds_orig, folds_test])
folds_corrected_test_appended = pd.concat([folds_corrected, folds_test])

In [82]:
folds_orig_test_appended.to_csv('5fold-test.csv', index=False)
folds_corrected_test_appended.to_csv('5fold-test-rev3.csv', index=False)